## Задание 1. Собираем данные с помощью Requests

In [49]:
import requests
import json
import csv
from pprint import pprint
import time
from time import sleep
from datetime import datetime

import os
from dotenv import load_dotenv
from pathlib import Path

import vk_api
import json

In [52]:
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)
user_id = os.getenv('VK_USER_ID')
access_token = os.getenv('VK_ACCES_TOKEN')
login_vk = os.getenv('VK_LOGIN')
pswd = os.getenv('VK_PSW')

In [24]:
url_country = 'https://api.vk.com/method/database.getCountries'
url_city = 'https://api.vk.com/method/database.getCities'
url_universities = 'https://api.vk.com/method/database.getUniversities'
url_faculties = 'https://api.vk.com/method/database.getFaculties'

In [77]:
def get_access_token():
    vk_session = vk_api.VkApi(
        login=login_vk, # не срабатывает через dotenv
        password=pswd)
    vk_session.auth()
    vk = vk_session.get_api()
    access_token = 0

    try:
        User = vk.users.get()
    except:
        print("Error")
    else:
        # print(f"\nHello {User[0]['first_name']}")
    
        with open('vk_config.v2.json', 'r') as data_file:
            data = json.load(data_file)
        
        for e in data['miaposta707@yandex.ru']:
            print(e)
    
        # for xxx in data[login]['token'].keys():
        #     for yyy in data[login]['token'][xxx].keys():
        #         access_token = data[login]['token'][xxx][yyy]['access_token']
    
        # print('=' * 85)
        # print(f"Твой ID {User[0]['id']}")
        # print('=' * 85)
        # print(f"Access_Token: {access_token}")
        # print('=' * 85)
    
        # os.remove('vk_config.v2.json')



    # url = f'https://oauth.vk.com/authorize?client_id={user_id}&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=friends&response_type=token&v=5.194'
    # res = requests.get(url)
    # print(res)

In [78]:
get_access_token()

LoginRequired: Login is required to auth

In [4]:
const_params = {'access_token': access_token, 'v': 5.131}

In [5]:
def get_items(r):
    items = r['response']['items']

    if r.get('error'):
        # print(f'Get item error!')
        return
    else:
        if len(items) < 1:
            # print(f'No {items} was found!')
            return
        else:
            return items

In [6]:
def get_country_id(code):
    params = {**const_params, 'code': code, 'count': 1}

    country = get_items(
        requests.get(url_country, params=params).json()
    )

    if not country: 
        print('No country was found!')
        return
    return country[0]['id']

In [7]:
get_country_id('RU')

1

In [8]:
def get_city(country_code, name):
    params = {**const_params,
                'country_id': get_country_id(country_code),
                'q': name,
                'count': 1
            }

    city = get_items(
        requests.get(url_city, params=params).json()
    )

    if not city: 
        print('No city was found!')
        return
    return city[0]

In [9]:
get_city('RU', 'томск')

{'id': 144,
 'title': 'Томск',
 'area': 'Городской округ Томск',
 'region': 'Томская область'}

In [10]:
def get_universities(country_code, city_name):
    params = {**const_params,
                'country_id': get_country_id(country_code),
                'city_id': get_city(country_code, city_name)['id'],
                'count': 100
            }

    universities = get_items(
        requests.get(url_universities, params=params).json()
    )
    universities_list = []

    if not universities: 
        print('No universities was found!')
        return
    for university in universities:
        university_id = university['id']
        university_name = university['title'].translate(dict.fromkeys(range(32)))
        # удаление управляющих символов из строки
        universities_list.append([university_id, university_name])

    return universities_list

In [11]:
get_universities('RU', 'Томск')

[[848, 'СибГМУ (ТМИ)'],
 [849, 'ТГАСУ'],
 [850, 'ТГПУ'],
 [851, 'ТГУ'],
 [852, 'ТУСУР (бывш. ТАСУР, ТИАСУР)'],
 [853, 'ТПУ'],
 [2403, 'ТВМИ'],
 [6255, 'Филиал РГСУ'],
 [13415, 'ТЭЮИ'],
 [18292, 'ТФ ВЭГУ'],
 [20715, 'ЗСФ РГУП (бывш. ЗСФ РАП)'],
 [23599, 'ЗСФ РГУИТП'],
 [35214, 'СГА (ТФ)'],
 [66758, 'ТИБ'],
 [75690, 'МГГУ им. Шолохова (ТФ)'],
 [118714, 'ТФ РАНХиГС (бывш. СибАГС ТФ)'],
 [118715, 'НГАВТ (ТФ)'],
 [118753, 'АПУ РФ (ТФ)'],
 [169809, 'ТВВКУС'],
 [170028, 'ТСХИ (ф) НГАУ'],
 [245701, 'ТДС'],
 [401125, 'ИФПМ СО РАН'],
 [461553, 'ТИИТ'],
 [499318, 'Томский ИПКР ФСИН России (бывш. ТФ Кузбасский институт ФСИН)'],
 [573625, 'ИМКЭС СО РАН'],
 [888642, 'ИСЭ СО РАН'],
 [1167989, 'ИХН СО РАН'],
 [1169980, 'НИИ фармакологии СО РАМН'],
 [1172216, 'НИИМГ СО РАМН'],
 [1174741, 'НИИ кардиологии СО РАМН'],
 [1176194, 'ИОА СО РАН'],
 [1188059, 'НИИЯФ при ТПУ'],
 [1192309, 'НИИ психического здоровья Томского НИМЦ']]

In [12]:
def get_faculties(university_id):
    universities_list = []
    params = {**const_params,
                'university_id': university_id,
                'count': 100
            }

    faculties = get_items(
        requests.get(url_faculties, params=params).json()
    )
    faculties_list = []

    if not faculties: 
        # print('No faculties was found!')
        return
    for faculty in faculties:
        faculties_list.append(faculty['title'])

    return faculties_list

In [13]:
get_faculties(848)

['Лечебный факультет',
 'Педиатрический факультет',
 'Фармацевтический факультет',
 'Факультет клинической психологии, психотерапии и социальной работы',
 'Факультет управления и экономики в здравоохранении',
 'Медико-биологический факультет',
 'Факультет высшего сестринского образования',
 'Факультет поведенческой медицины и менеджмента',
 'Факультет повышения квалификации и профессиональной переподготовки специалистов',
 'Отдел ординатуры']

In [14]:
def get_data_university(data):
    try:
        university_id = data[0]
        university_name = data[1]
    except:
        print('no data')
    try:
        faculties_list = get_faculties(university_id)
    except:
        print('error in faculties_list')
    result = {
        'id': university_id, 
        'name': university_name,
        'faculties': faculties_list
    }
    return result


In [23]:
data = [848, 'СибГМУ (ТМИ)']
get_data_university(data)

{'id': 848,
 'name': 'СибГМУ (ТМИ)',
 'faculties': ['Лечебный факультет',
  'Педиатрический факультет',
  'Фармацевтический факультет',
  'Факультет клинической психологии, психотерапии и социальной работы',
  'Факультет управления и экономики в здравоохранении',
  'Медико-биологический факультет',
  'Факультет высшего сестринского образования',
  'Факультет поведенческой медицины и менеджмента',
  'Факультет повышения квалификации и профессиональной переподготовки специалистов',
  'Отдел ординатуры']}

{
  "cities": [
    {   
        "id": <ID города>,
        "name": <Название города>,
        "universites": [
            {
              "id": <ID ВУЗа>, 
              "name": <Название ВУЗа>
              "faculties": [<Название факультета>, …]
            },

In [52]:
def get_faculties_universities(country_code, city_name):
    results = {}
    cities_list = []
    universities = []

    universities_list = get_universities(country_code, city_name)

    for university in universities_list:
        universities.append(get_data_university(university))

    return universities

In [53]:
get_faculties_universities('RU', 'Биробиджан')

[{'id': 393,
  'name': 'ПГУ им. Шолом-Алейхема (бывш. ДВГСГА, БГПИ)',
  'faculties': ['Факультет дополнительного профессионального образования и подготовки аспирантов (Повышения квалификации)',
   'Факультет математики, информационных технологий и техники',
   'Факультет таможенного дела и геосреды',
   'Факультет филологии, истории и журналистики',
   'Факультет экономики, управления и права',
   'Факультет педагогики и психологии',
   'Факультет социально-культурной деятельности и сервиса',
   'Факультет географии и природопользования',
   'Факультет экономики и управления',
   'Факультет сервиса, рекламы и социальной работы',
   'Факультет дизайна и искусств',
   'Факультет иностранных языков',
   'Факультет математики и информатики',
   'Факультет социальной педагогики и психологии',
   'Факультет коррекционной педагогики и начального образования',
   'Художественно-графический факультет',
   'Филолого-исторический факультет']},
 {'id': 3576,
  'name': 'БФ ХГАЭиП',
  'faculties': [

In [54]:
get_faculties_universities('RU', 'Орск')

error in faculties_list


UnboundLocalError: local variable 'faculties_list' referenced before assignment